# **Astronomy RAG Model: AI-Powered Knowledge Retrieval** 

This project aims to develop a **Retrieval-Augmented Generation (RAG) model** trained on an **astronomy book** to provide intelligent and context-aware responses to user queries. By integrating **Large Language Models (LLMs)** with a **vector-based retrieval system**, the model enhances **fact-based** answering by leveraging **retrieved text from the book** rather than relying solely on generative capabilities.  

## **🔹 Key Features**  
- **PDF/Text Loader** – Extracts information from an astronomy book.  
- **FAISS Vector Search** – Efficient document retrieval using embeddings.  
- **LLM Integration** – Uses GPT or Hugging Face models for intelligent responses.  
- **Accurate Scientific Answers** – Provides fact-based insights on black holes, galaxies, space-time, and more.  
- **Interactive User Interface** – Command-line or Jupyter Notebook for easy querying.  

## **Workflow**  
1. **Load** the astronomy book into the system.  
2. **Chunk & Embed** the text using **FAISS** and OpenAI/Hugging Face embeddings.  
3. **Retrieve** relevant passages based on user queries.  
4. **Generate responses** using an LLM, grounded in the retrieved data.  

## **Goal**  
To create an **AI-powered astronomy assistant** capable of answering **scientific queries** with high accuracy, enhancing research and education in astrophysics. 

# Importing Libraries

In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_openai import OpenAI
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_huggingface import ChatHuggingFace,HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from dotenv import load_dotenv

In [2]:
import os

## What is LangChain? 

LangChain is a **framework** designed to help developers **build applications** powered by **large language models (LLMs)** like OpenAI’s GPT, Hugging Face models, and other AI models. It provides **modular tools** for integrating language models with **external data sources, retrieval-augmented generation (RAG), memory, and reasoning**.  

---

## Key Features of LangChain  

1. **LLM Integration** – Easily connects with OpenAI, Hugging Face, and other models.  
2. **Retrieval-Augmented Generation (RAG)** – Uses **document retrieval** to improve model responses.  
3. **Memory** – Stores conversation history for **context-aware** interactions.  
4. **Chains** – Sequences multiple steps together (e.g., prompt → LLM → post-processing).  
5. **Agents** – Uses AI to decide **which tool** (search, database, API) to call at runtime.  
6. **Tools & Plugins** – Integrates with **APIs, databases, embeddings, and vector stores**.  

---

## Why Use LangChain? 
**Customizable** – You can fine-tune behavior and data sources.  
**Scalable** – Supports small apps and **enterprise-level AI**.  
**Multi-Modal** – Works with text, documents, images, and even structured data.  

---

## Example Use Cases 
- **Chatbots & Virtual Assistants** 
- **Legal or Financial Document Analysis**   
- **RAG Systems (e.g., PDF Q&A with Vector Search)**   
- **Automated Report Generation** 
- **AI-powered Search Engines**  

---

# Importing ChatHuggingFace for interating with HuggingFace LLMS

# **What is Hugging Face?** 

Hugging Face is an **AI research organization and open-source platform** that provides **pretrained models, datasets, and tools** for **Natural Language Processing (NLP), Computer Vision (CV), and Machine Learning (ML)**.  

It is best known for the **Transformers** library, which allows developers to use **state-of-the-art models** like GPT, BERT, Llama, and more.

---

##  Key Features of Hugging Face 

1. **Transformers** – A Python library with thousands of **pretrained NLP and CV models**.  
2. **Datasets** – A collection of **high-quality datasets** for AI training.  
3. **Model Hub** – A cloud-based **repository** to store and share AI models.  
4. **Inference API** – Run AI models in the cloud **without setting up infrastructure**.  
5. **AutoTrain** – No-code tool for **fine-tuning models** easily.  
6. **Accelerate** – Optimized tools for **faster model training** and deployment.  

---

## **Why Use Hugging Face?**  
**Pretrained Models** – Save time with state-of-the-art models.  
**Easy Fine-Tuning** – Train models with your **custom data**.  
**Cloud & API Support** – Deploy models instantly with APIs.  
**Large Community** – Thousands of researchers and developers contribute.  

---

## **Example Use Cases**  
- **Chatbots & Virtual Assistants**  
- **Text Summarization**   
- **Image Generation (Stable Diffusion)**  
- **Speech-to-Text (ASR)** 
- **Sentiment Analysis**  
- **Legal & Financial Document Processing**  

---



In [3]:
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint

In [4]:
llm = HuggingFacePipeline.from_model_id(
    model_id='TinyLlama/TinyLlama-1.1B-Chat-v1.0',
    task='text-generation',
    pipeline_kwargs = dict(
        temperature = 1,
        max_new_tokens = 100
    )
)

Device set to use mps:0


In [5]:
chat_model = ChatHuggingFace(llm=llm)

In [6]:
import ipywidgets as widgets
from IPython.display import display,clear_output

In [7]:
def text_splitter(documents):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=50)
    docs = text_splitter.split_documents(documents)
    return docs

In [8]:
def load_document():
    loader = PyPDFLoader("/Users/shivendragupta/Desktop/Langchain Models/RAG demo/06. Astronomy An overview Autor Poincare.pdf")
    documents = loader.load()
    return text_splitter(documents)


In [9]:
def init_retreiver(docs):
    vector_store = FAISS.from_documents(docs , HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2'))

    return vector_store.as_retriever()

In [10]:
def get_retreived_docs(retreiver,query):
    return retreiver.get_relevant_documents(query)

# Defining the Widgets

In [11]:
query_input = widgets.Text(placeholder='Enter your query...')
submit_button = widgets.Button(description='Search')
output_area = widgets.Output()
response_box = widgets.Textarea(  # Scrollable output box
    value="",
    placeholder="Response will appear here...",
    layout=widgets.Layout(width="100%", height="300px")  # Adjust height as needed
)

HTML(value='<b>Generating response...</b>')

HTML(value='<b>Generating response...</b>')

In [12]:
def final_prompt(retrived_docs,query):
    retre = '\n'.join([doc.page_content for doc in retrived_docs])
    prompt = f"Based on following text , answer the question : {query} \n\n {retre}"
    return prompt

In [13]:
docs = load_document()
reteiver = init_retreiver(docs)

/var/folders/qx/720g_1z55gg_fwcvggbhs3_w0000gn/T/ipykernel_76123/702652017.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  vector_store = FAISS.from_documents(docs , HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2'))


# Handling the click of submit button

In [14]:
def handle_submit(_):
    loading_spinner = widgets.HTML(value="<b>Generating response...</b>")
    display(loading_spinner)
    retrived_docs = get_retreived_docs(reteiver,query_input.value)
    prompt = final_prompt(retrived_docs,query_input.value)
    response = chat_model.invoke(prompt) 

   

    with output_area:
        clear_output()
        clean_response = response.content.split("</s>")[-1]
        

        res = clean_response.split("</s>")[-1]
        response_box.value = res
        loading_spinner = ""
        print("Generated Output")

In [15]:
submit_button.on_click(handle_submit)

In [16]:
loading_spinner = widgets.HTML(value="<b>Generating response...</b>")
display(widgets.VBox([query_input, submit_button,output_area,response_box,loading_spinner]))